# [MAC0417] Visão e Processamento de Imagens - Exercício Programa 3.2
*22 de Janeiro de 2021*

## Introdução 
A segunda parte do Exercício Programa 3 

Integrantes do grupo:

| Nome | NUSP |
|------|------|
| Daniela Favero | 10277443 |
| Mateus Barbosa | 8993368 |
| Vitor Guidi | 8038091 |

A base de dados e os metadados se encontram no nosso repositório do [GitHub](https://github.com/danigfavero/Visao-computacional-e-processamento-de-imagens).

## Preparação do ambiente de programação
### Importando as bibliotecas necessárias para a implementação dos algoritmos

In [15]:
import matplotlib.pyplot as plt
import itertools
import numpy as np
import os
from skimage import io
from skimage.exposure import equalize_hist
from matplotlib.pyplot import imshow

from skimage.measure import perimeter
from scipy.stats import mode
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

## Feature Extraction
A partir da Feret Box, extraímos algumas features relevantes para a classificação da imagem: a razão perímetro / área e a razão de pixels pretos por pixels brancos na imagem.

In [16]:
def get_perimeter_area(img):
    x, y = img.shape
    area = x*y
    per = perimeter(img)
    return per/area

def get_black_white_pixels_ratio(img):
    x, y = img.shape
    area = x*y
    white = np.count_nonzero(img)
    black = area - white
    if white == 0:
        white = 1 # evita divisão por zero
    return black/white

FEATURE_FUNCTIONS = [get_perimeter_area, get_black_white_pixels_ratio]

As funções `get_perimeter_area` e `get_black_white_pixels_ratio` implementam a extração da razão do perímetro pela área e da quantidade de pixels brancos por pixels pretos, respectivamente. `FEATURE_FUNCTIONS` é uma lista com todas as funções de extração de features, a serem usadas a seguir.

In [17]:
def extract_features(img):
    img = img/255 # normaliza branco de 255 para 1
    
    features = []
    for f in FEATURE_FUNCTIONS:
        features.append(f(img))
    return features

def extract_all_features(input_directory):
    features = []
    classes = []
    for subdir in os.scandir(input_directory):
        for file in os.scandir(subdir):
            path = file.path
            for _ in range(3): # evitando erros de conexão (3 tentativas)
                try:
                    img = io.imread(path).astype(np.uint8)
                    class_name = path.split("/")[1]
                    classes.append(class_name)
                    features.append(extract_features(img))
                except ConnectionResetError:
                    continue
                break
    return features, classes

As função `extract_features` extrai todas as features de uma única imagem; ela é chamada pela função `extract_all_features` para extrair o vetor de features de todas as imagens.

## Classificação

In [19]:
features, classes = extract_all_features("feret/")
feature_train, feature_test, class_train, class_test = train_test_split(features, classes)
clf = tree.DecisionTreeClassifier()
clf.fit(feature_train, class_train)
predicted = clf.predict(feature_test)
confusion = confusion_matrix(class_test, predicted)

Finalmente, separamos as features e as classes em conjuntos de teste e de treino, a fim de treinar o algoritmo de _machine learning_ e testá-lo a seguir. A matriz de confusão é obtida a partir da classe predita e da classe real para os objetos do conjunto de teste. 

In [20]:
print(confusion)

[[16  1  8  3]
 [ 4  0  0  2]
 [ 6  3 12  6]
 [ 7  2  4 13]]
